# Tutorial for HP1alpha dimer and dsDNA

In [1]:
import sys
sys.path.append('../..')

import numpy as np
import pandas as pd
from openabc.forcefields.parsers import MOFFParser, MRGdsDNAParser
from openabc.forcefields import MOFFMRGModel
from openabc.utils.insert import insert_molecules
import simtk.openmm as mm
import simtk.openmm.app as app
import simtk.unit as unit
import os

# set simulation platform
platform_name = 'CPU'

We show the tutorial of how to run simulation for a system including n1 HP1alpha dimers and n2 dsDNA with MOFF. For simplicity, here we set both n1 and n2 as 5. 

In [2]:
# parse atomistic structure protein_AA.pdb and dsDNA_AA.pdb
hp1alpha_dimer_parser = MOFFParser.from_atomistic_pdb('input-pdb/hp1a.pdb', 'hp1alpha_dimer_CA.pdb')
dsDNA_parser = MRGdsDNAParser.from_atomistic_pdb('input-pdb/all_atom_200bpDNA.pdb', 'dsDNA_CG.pdb')

Parse molecule with default settings.
Get native pairs with shadow algorithm.
Parse molecule with default settings.


In [3]:
# remove native pairs with atoms in disordered regions
old_native_pairs = hp1alpha_dimer_parser.native_pairs.copy()
new_native_pairs = pd.DataFrame(columns=old_native_pairs.columns)
cd1 = np.arange(16, 72)
csd1 = np.arange(114, 176)
n_atoms_per_hp1alpha_dimer = len(hp1alpha_dimer_parser.atoms.index)
print(f'There are {n_atoms_per_hp1alpha_dimer} CA atoms in each HP1alpha dimer.')
cd2 = cd1 + int(n_atoms_per_hp1alpha_dimer/2)
csd2 = csd1 + int(n_atoms_per_hp1alpha_dimer/2)
for i, row in old_native_pairs.iterrows():
    a1, a2 = int(row['a1']), int(row['a2'])
    if a1 > a2:
        a1, a2 = a2, a1
    flag1 = ((a1 in cd1) and (a2 in cd1)) or ((a1 in csd1) and (a2 in csd1))
    flag2 = ((a1 in cd2) and (a2 in cd2)) or ((a1 in csd2) and (a2 in csd2))
    flag3 = ((a1 in csd1) and (a2 in csd2))
    if flag1 or flag2 or flag3:
        new_native_pairs.loc[len(new_native_pairs.index)] = row
hp1alpha_dimer_parser.native_pairs = new_native_pairs
hp1alpha_dimer_parser.parse_exclusions() # update exclusions based on the new native pairs

There are 382 CA atoms in each HP1alpha dimer.


In [4]:
# prepare initial configuration with n1 proteins and n2 dsDNA
a, b, c = 100, 100, 100
n1, n2 = 2, 2
if not os.path.exists('start.pdb'):
    insert_molecules('hp1alpha_dimer_CA.pdb', 'tmp.pdb', n_mol=n1, box=[a, b, c])
    insert_molecules('dsDNA_CG.pdb', 'start.pdb', n_mol=n2, existing_pdb='tmp.pdb', box=[a, b, c])


Box size is 1000*1000*1000 angstrom^3. 
Successfully inserted 2 molecules.
Box size is 1000*1000*1000 angstrom^3. 
Successfully inserted 2 molecules.


/home/gridsan/sliu/.conda/envs/py39/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/gridsan/sliu/.conda/envs/py39/lib/python3.9/site-packages/MDAnalysis/topology/PDBParser.py:328: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/home/gridsan/sliu/.conda/envs/py39/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'elements' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"


In [5]:
protein_dna = MOFFMRGModel()
for i in range(n1):
    protein_dna.append_mol(hp1alpha_dimer_parser)
for i in range(n2):
    protein_dna.append_mol(dsDNA_parser)
protein_dna.native_pairs.loc[:, 'epsilon'] = 6.0
top = app.PDBFile('start.pdb').getTopology()
protein_dna.create_system(top)
salt_conc = 82*unit.millimolar
temperature = 300*unit.kelvin
protein_dna.add_protein_bonds(force_group=1)
protein_dna.add_protein_angles(force_group=2)
protein_dna.add_protein_dihedrals(force_group=3)
protein_dna.add_native_pairs(force_group=4)
protein_dna.add_dna_bonds(force_group=5)
protein_dna.add_dna_angles(force_group=6)
protein_dna.add_dna_fan_bonds(force_group=7)
protein_dna.add_contacts(force_group=8)
protein_dna.add_elec_switch(salt_conc, temperature, force_group=9)
protein_dna.save_system('system.xml')
friction_coeff = 0.01/unit.picosecond # use small friction coefficient to accelerate dynamics
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
init_coord = app.PDBFile('start.pdb').getPositions()
protein_dna.set_simulation(integrator, platform_name, init_coord=init_coord)
protein_dna.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output.dcd'
protein_dna.add_reporters(output_interval, output_dcd)
protein_dna.simulation.context.setVelocitiesToTemperature(temperature)
protein_dna.simulation.step(500)

Add protein bonds.
Add protein angles.
Add protein dihedrals.
Add native pairs.
Add DNA bonds.
Add DNA angles.
Add DNA fan bonds.
Add protein and DNA nonbonded contacts.
Add protein and DNA electrostatic interactions with distance-dependent dielectric and switch.
Add electrostatic interactions between native pair atoms.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,-1717.118716664791,2610.802672803358,893.6839561385668,133.93362593725897,0
200,2.0000000000000013,-2227.2850887263703,3163.935468291967,936.6503795655967,162.30922157162942,53.6
300,2.99999999999998,-2192.709073951711,3185.1528659963383,992.4437920446271,163.39766959456946,53.5
400,3.9999999999999587,-2125.4528550986556,3209.1167577067954,1083.6639026081398,164.62701218019774,52.9
500,4.999999999999938,-2097.5171801568854,3206.041206607052,1108.5240264501667,164.46923705807347,53.1
